# 3 Tactics to Improve your Cluster Analysis

We will use the Mall Customers dataset, which is available for download
[from Kaggle](https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python)
for example.

The dataset contains information about 200 mall customers. For each of the 200
mall customers, the dataset includes
- a `CustomerID`
- the customer's `Gender`
- the customer's `Age`
- the customer's `Annual Income` (in thousands of USD)
- a `Spending Score` (between 1 and 100) assigned by the mall to each customer
  based on the customer's spending habits.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.optimize import linear_sum_assignment
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.preprocessing import StandardScaler

In [ ]:
pd.set_option("display.max_columns", 100)

## Read the Data

In [ ]:
data = pd.read_csv("../data/mall_customers.csv")

In [ ]:
data.head()

We are not going to use the `CustomerID` for the purpose of clustering.

In [ ]:
data.drop("CustomerID", axis=1, inplace=True)

In [ ]:
data.head()

To keep things simple, we will also ignore the only categorical variable in the
dataset i.e., the customer's `Gender`.

In [ ]:
data.drop("Gender", axis=1, inplace=True)

In [ ]:
data.describe()

We standardize the variables in the dataset.

In [ ]:
standardized_data = StandardScaler().fit_transform(data)

In [ ]:
standardized_data.mean(axis=0)

In [ ]:
standardized_data.var(axis=0)

## Utilities

In [ ]:
# random seed for sklearn estimators
RANDOM_STATE = 42

In [ ]:
def get_clusterer_score(clusterer, data, score_function):
    """Evaluates a clusterer with respect to a score."""
    labels = clusterer.labels_
    return score_function(data, labels)

In [ ]:
def make_cluster_analysis(
    clusterer,
    parameter_name,
    parameter_values,
    data,
    score_function,
    larger_is_better=True,
):
    """Analyzes the performance of a clusterer with respect to a score and
    as a function of its (main) tuning parameter."""
    scores = []
    for parameter_value in parameter_values:
        clusterer.set_params(**{parameter_name: parameter_value})
        clusterer = clusterer.fit(data)
        score = get_clusterer_score(clusterer, data, score_function)
        scores.append(score)
    return {
        "parameter_name": parameter_name,
        "parameter_values": parameter_values,
        "data": data,
        "score_function": score_function,
        "scores": scores,
        "best_parameter_value": parameter_values[np.argmax(scores)]
        if larger_is_better
        else parameter_values[np.argmin(scores)],
    }

In [ ]:
def plot_analysis(clusterer_analysis, path=None):
    parameter_name = clusterer_analysis["parameter_name"]
    parameter_values = clusterer_analysis["parameter_values"]
    scores = clusterer_analysis["scores"]
    score_function_name = clusterer_analysis["score_function"].__qualname__

    plt.plot(parameter_values, scores)
    ax = plt.gca()
    ax.set_xlabel(parameter_name)
    ax.set_ylabel(score_function_name)
    ax.set_xticks(parameter_values)
    ax.set_xticklabels(parameter_values)
    ax.set_yticks([])
    ax.set_yticklabels([])
    plt.xticks(rotation=45)
    plt.tight_layout()

    if path:
        plt.savefig(path, dpi=300)

In [ ]:
def plot_clustering(clusterer, data, path=None):
    data_copy = data.copy()
    data_copy["labels"] = clusterer.labels_

    n_colors = clusterer.labels_.max() - clusterer.labels_.min() + 1
    sns.pairplot(data_copy, hue="labels", palette=sns.color_palette("husl", n_colors))
    plt.tight_layout()

    if path:
        plt.savefig(path, dpi=300)

In [ ]:
def cross_tabulate_clusterers(clusterer0, clusterer1, clusterer0_name, clusterer1_name):
    """Creates a cross-tabulation of the clusters found by two clusterers
    that allows to evaluate their agreement.
    """
    if clusterer1.labels_.max() >= clusterer0.labels_.max():
        clusterer0, clusterer1 = clusterer1, clusterer0
        clusterer0_name, clusterer1_name = clusterer1_name, clusterer0_name

    cross_tab = pd.crosstab(
        clusterer0.labels_,
        clusterer1.labels_,
        rownames=[clusterer0_name],
        colnames=[clusterer1_name],
    )

    optimal_column_permutation = linear_sum_assignment(-cross_tab)[1]
    return cross_tab.loc[:, optimal_column_permutation]

## Visualize the Data

In [ ]:
_ = sns.pairplot(data)
plt.savefig("pairplot.jpeg", dpi=300)

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=RANDOM_STATE).fit(standardized_data)

In [ ]:
plot_clustering(kmeans, data, path="kmeans-clustering.jpeg")

## Tactic 1: Algorithm Tuning

Here, we use k-means clustering with the Silhouette Score.

In [ ]:
kmeans_analysis_silhouette = make_cluster_analysis(
    clusterer=KMeans(random_state=RANDOM_STATE),
    parameter_name="n_clusters",
    parameter_values=np.arange(2, 16),
    data=standardized_data,
    score_function=silhouette_score,
)

In [ ]:
plot_analysis(kmeans_analysis_silhouette, path="kmeans-silhouette-tuning.jpeg")

In [ ]:
kmeans_analysis_silhouette["best_parameter_value"]

The Silhouette score suggests 6 (or 10?) clusters. Accordingly, we fit a k-means
model with 6 clusters.

In [ ]:
kmeans_silhouette = KMeans(
    n_clusters=kmeans_analysis_silhouette["best_parameter_value"],
    random_state=RANDOM_STATE,
).fit(standardized_data)

Here is a plot of the clusters.

In [ ]:
plot_clustering(kmeans_silhouette, data, path="kmeans-silhouette-clustering.jpeg")

Let's look at some statistics for these clusters.

In [ ]:
clustered_data = data.copy()
clustered_data["label"] = kmeans_silhouette.labels_
clustered_data.groupby("label").describe()

What about choosing the second-best parameter value i.e., `n_clusters = 10`?

In [ ]:
kmeans_silhouette10 = KMeans(
    n_clusters=10,
    random_state=RANDOM_STATE,
).fit(standardized_data)

In [ ]:
plot_clustering(kmeans_silhouette10, data, path="kmeans-silhouette10-clustering.jpeg")

In [ ]:
cross_tabulate_clusterers(
    kmeans_silhouette, kmeans_silhouette10, "kmeans_6", "kmeans_10"
)

It seems that with `n_clusters = 10`, some of the existing clusters are further
broken down.

## Tactic 2: Sensitivity Analysis

What if we used the Calinski-Harabasz score instead of the Silhouette score?

In [ ]:
kmeans_analysis_ch = make_cluster_analysis(
    clusterer=KMeans(random_state=RANDOM_STATE),
    parameter_name="n_clusters",
    parameter_values=np.arange(2, 16),
    data=standardized_data,
    score_function=calinski_harabasz_score,
)

In [ ]:
plot_analysis(kmeans_analysis_ch, path="kmeans-ch-tuning.jpeg")

In [ ]:
kmeans_analysis_ch["best_parameter_value"]

The Calinski-Harabasz score suggests 11 clusters, although 6 is still a highly
scored parameter value.

In [ ]:
kmeans_ch = KMeans(
    n_clusters=kmeans_analysis_ch["best_parameter_value"], random_state=RANDOM_STATE
).fit(standardized_data)

In [ ]:
cross_tabulate_clusterers(
    kmeans_silhouette, kmeans_ch, "kmeans_silhouette", "kmeans_ch"
)

It looks like the model tuned using the Calinski-Harabasz score provides a
more fine-grained clustering than the model tuned using the Silhouette
score.

Some clusters found by the latter are further split by the former (e.g.,
cluster 4 of the model tuned using the Silhouette score is split into two
clusters - cluster 5 and 7 - by the model tuned using the Calinski-Harabasz
score).

What about the Davies-Bouldin score?

In [ ]:
kmeans_analysis_db = make_cluster_analysis(
    clusterer=KMeans(random_state=RANDOM_STATE),
    parameter_name="n_clusters",
    parameter_values=np.arange(2, 16),
    data=standardized_data,
    score_function=davies_bouldin_score,
    larger_is_better=False,
)

In [ ]:
plot_analysis(kmeans_analysis_db, path="kmeans-db-tuning.jpeg")

In [ ]:
kmeans_analysis_db["best_parameter_value"]

The Davies-Bouldin score suggests again 6 clusters, though 10 is again scored
very favorably.

In [ ]:
kmeans_db = KMeans(
    n_clusters=kmeans_analysis_db["best_parameter_value"], random_state=RANDOM_STATE
).fit(standardized_data)

In [ ]:
cross_tabulate_clusterers(
    kmeans_silhouette, kmeans_db, "kmeans_silhouette", "kmeans_db"
)

The clusters found by the model tuned using the Davies-Bouldin score are
exactly the same as the ones found by the model tuned using the Silhouette
score.

## Tactic 3: Consensus Analysis

What if we used another clustering algorithm such as agglomerative clustering?

In [ ]:
agglomerative_analysis_silhouette = make_cluster_analysis(
    clusterer=AgglomerativeClustering(),
    parameter_name="n_clusters",
    parameter_values=np.arange(2, 16),
    data=standardized_data,
    score_function=silhouette_score,
)

In [ ]:
plot_analysis(
    agglomerative_analysis_silhouette, path="agglomerative-silhouette-tuning.jpeg"
)

In [ ]:
agglomerative_analysis_silhouette["best_parameter_value"]

With the Calinski-Harabasz score, Agglomerative Clustering finds 6 clusters.

Are these the same as the ones found by k-means?

In [ ]:
agglomerative_silhouette = AgglomerativeClustering(
    n_clusters=agglomerative_analysis_silhouette["best_parameter_value"]
).fit(standardized_data)

In [ ]:
cross_tabulate_clusterers(
    kmeans_silhouette,
    agglomerative_silhouette,
    "kmeans_silhouette",
    "agglomerative_silhouette",
)

What if we tuned Agglomerative Clustering using other score functions?

In [ ]:
agglomerative_analysis_ch = make_cluster_analysis(
    clusterer=AgglomerativeClustering(),
    parameter_name="n_clusters",
    parameter_values=np.arange(2, 16),
    data=standardized_data,
    score_function=calinski_harabasz_score,
)

In [ ]:
plot_analysis(agglomerative_analysis_ch, path="agglomerative-ch-tuning.jpeg")

In [ ]:
agglomerative_ch = AgglomerativeClustering(
    n_clusters=agglomerative_analysis_ch["best_parameter_value"]
).fit(standardized_data)

In [ ]:
cross_tabulate_clusterers(
    kmeans_silhouette, agglomerative_ch, "kmeans_silhouette", "agglomerative_ch"
)

In [ ]:
agglomerative_analysis_db = make_cluster_analysis(
    clusterer=AgglomerativeClustering(),
    parameter_name="n_clusters",
    parameter_values=np.arange(2, 16),
    data=standardized_data,
    score_function=davies_bouldin_score,
    larger_is_better=False,
)

In [ ]:
plot_analysis(agglomerative_analysis_db, path="agglomerative-db-tuning.jpeg")

In [ ]:
agglomerative_db = AgglomerativeClustering(
    n_clusters=agglomerative_analysis_db["best_parameter_value"]
).fit(standardized_data)

In [ ]:
cross_tabulate_clusterers(
    kmeans_silhouette, agglomerative_db, "kmeans_silhouette", "agglomerative_db"
)

All in all, it seems there is a large amount of agreement between the
results generated by these two algorithms.

It appears that there may in fact be 6 clusters in this dataset, with
potentially additional sub-clusters of interest.

## Remarks

We have to be careful, however.

In general, scoring functions do not automatically prevent us from making
foolish choices.

For example, here is what the Calinski-Harabasz score looks like for very large
values of `n_clusters` in k-means.

In [ ]:
kmeans_ch_large_n_clusters = make_cluster_analysis(
    clusterer=KMeans(random_state=RANDOM_STATE),
    parameter_name="n_clusters",
    parameter_values=np.array(
        [2, 6, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192]
    ),
    data=standardized_data,
    score_function=calinski_harabasz_score,
)

In [ ]:
plot_analysis(kmeans_ch_large_n_clusters, path="kmeans-ch-tuning-large-n-clusters.jpeg")